In [1]:
import param as pm
import panel as pn

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
class Wallet(pm.Parameterized):
    total_fund = pm.Number(0, bounds=(0, None), doc="wallet amount in USD")
    total_ctr = pm.Number(0, bounds=(0, None), doc="Amount of CTR tokens in wallet")
    
    def __init__(self, **params):
        super().__init__(**params)
    
    def add_funds(self, amount):
        self.total_fund += amount
        
    def add_ctr(self, ctr):
        self.total_ctr += ctr
        
    def return_funds(self):
        return self.total_fund
    
    def return_ctr(self):
        return self.total_ctr

In [3]:
class OldGrowthForestUnit(pm.Parameterized):
    """OGFU"""
    ogfu_name = pm.String('', doc="Name of particular OGFU")
    tree_century_count = pm.Number(32000, bounds=(0,1e6), doc="TCC")
    cost_for_protection = pm.Number(320000, bounds=(0, 1e6), doc="Cost for Protection")
    token_multiplier = pm.Number(1, bounds=(0, 10), doc="TM")
    staked_funds = pm.Number(0, bounds=(0, None), doc="The amount staked in a respective OGFU in CTR")
    staked_amount = pm.Number(0, bounds=(0, None), doc="The amount released after an OGFU is fully staked")
    approved = pm.Boolean(False, doc="Boolean that determines whether OGFU has been approved by Centree fund")
    full_stake = pm.Boolean(False, doc="Boolean that determines when the OGFU is fully staked")
    
    def __init__(self, **params):
        super().__init__(**params)
    
    def check_full_stake(self):
        # Assume OGFU is fully staked at 50000 CTR
        if (self.staked_funds >= 50000):
            self.full_stake = True
        else: 
            self.full_stake = False

    def name_ogfu(self, name):
        """Name"""
        self.ogfu_name = name
    
    def get_protection_cost_per_century(self):
        """PCC"""
        return self.cost_for_protection / self.tree_century_count
    
    def get_ctr_tokens_issued(self):
        """CTR Tokens Issued"""
        return self.tree_century_count * self.token_multiplier
    
    def stake(self, wallet: Wallet, ctr, total_ctr):
        """Community members vote to protect a particular OGFU by “staking” their CTR against particular
        Tree Centuries in that OGFU. OGFUs can only be staked against when their PCC is lower than 
        FundEquity/TotalCTRIssued. Minimum stake is 100CTR"""
        pcc = self.get_protection_cost_per_century()
        if (ctr > 100) and (pcc < total_ctr):
            # Remove the funded amount from user's wallet & add it to OGFU staked amount
            wallet.add_ctr(-ctr)
            self.staked_funds += ctr
            self.check_full_stake()
            
        return wallet
        
    def generate_revenue(self):
        """A funded OGFU will periodically generate revenue for later distribution"""
        pass

In [4]:
class Centree(pm.Parameterized):
    """Centree"""
    approved_protection_cost_per_century = pm.Number(500, bounds=(0, 10000), doc="APCC")
    centree_general_fund = pm.Number(0, bounds=(0, None), doc="CGF")
    total_ctr = pm.Number(0, bounds=(0, None), doc="Total CTR generated by Fund")
    #ogfu_dict = pm.Dict(doc="Dictionary to keep track of OGFUs")
    ogfu_dict = {}
    
    def __init__(self, **params):
        super().__init__(**params)
    
    def get_total_ctr_issued(self):
        """Returns total CTR issued in Centree General Fund"""
        return self.total_ctr
    
    def generate(self, ctr_generated):
        """Creates CTR to add to the existing pool of CTR"""
        self.total_ctr += ctr_generated
    
    def convert_funds_to_ctr(self, funds):
        """Functions that returns CTR equivalent to funds provided"""
        # Assume 1$ => 10 CTR
        ctr = funds * 10
        # Remove CTR from total CTR generated by centree general fund
        if (self.total_ctr < ctr):
            ctr_diff = (ctr - self.total_ctr)/10
            total_ctr_purchased = self.total_ctr
            self.total_ctr = 0
            return total_ctr_purchased, ctr_diff
        self.total_ctr -= ctr
        ctr_diff = 0
    
        return ctr, ctr_diff
    
    def approve(self, ogfu):
        """When an OGFU is approved by Centree for protection, its TCC & PCC are assessed 
        & CC*TM new CTR tokens are issued"""
        # Approve OGFU & generate CTR based on OFGU's TCC
        ogfu.approved = True
        ctr_generated = ogfu.get_ctr_tokens_issued()
        self.generate(ctr_generated)
        self.ogfu_dict[ogfu.ogfu_name] = ogfu
        
        return ogfu
    
    def buy(self, wallet, funds):
        """A wallet can exchange funds for CTR"""
        # Take funds from the wallet's funds & add CTR tokens in return
        self.centree_general_fund += funds
        wallet.add_funds(-funds)
        ctr_generated, ctr_diff = self.convert_funds_to_ctr(funds)
        wallet.add_ctr(ctr_generated)
        wallet.add_funds(ctr_diff)
        
        return wallet

In [ ]:
def wallet():
    
    wallet = Wallet()
    wallet.add_funds(50000)
    
    return wallet

In [ ]:
def centree():
    
    centree_fund = Centree()
    
    return centree_fund

In [ ]:
def ogfu_1():
    
    ogfu1 = OldGrowthForestUnit()
    ogfu1.name_ogfu('OGFU 1')
    
    return ogfu1

In [ ]:
def ogfu_2():
    
    ogfu2 = OldGrowthForestUnit()
    ogfu2.name_ogfu('OGFU 2')
    
    return ogfu2

In [ ]:
def test_approval(ogfu_1, ogfu_2, centree):
    # Initially, both OGFUs are not approved yet for Centree protection
    assert ogfu1.approved == False
    assert ogfu2.approved == False
    
    # We manually approve both OGFUs
    ogfu1 = centree.approve(ogfu1)
    ogfu2 = centree.approve(ogfu2)
    
    # Then check whether they have now been approved
    assert ogfu1.approved == True
    assert ogfu2.approved == True

In [ ]:
def test_buy(wallet, ogfu_1, ogfu_2, centree):
    # We first approve both OGFUs in order to generate CTR tokens in Centree fund
    ogfu1 = centree.approve(ogfu1)
    ogfu2 = centree.approve(ogfu2)
    
    # Check the amount of CTR generated, each OGFU generates 32000 CTR
    assert centree.get_total_ctr_issued() == 64000
    
    # User wallet purchases 1000$ worth of CTR
    wallet = centree.buy(wallet, 2000)
    
    # Check that user receives correct amount of CTR (fixed exchange rate 1$ => 10CTR)
    assert wallet.return_ctr() == 20000
    
    # Check that the correct amount was deducted from Centree CTR reserve (64000-20000=44000)
    assert centree.total_ctr() == 44000

In [ ]:
def test_stake(wallet, ogfu1, centree):
    # Ensure that Centree fund's CTR collection is not empty
    centree.generate(100000)
    
    # Wallet purchases CTR
    wallet = centree.buy(wallet, 5000)
    assert wallet.return_ctr() == 50000
    
    # Stake 5000CTR into OGFU 1
    total_ctr_fund = centree.get_total_ctr_issued()
    wallet = ogfu1.stake(wallet, 5000, total_ctr_fund)
    
    # Check whether the staking mechanism was successful
    assert ofgu1.staked_funds == 5000